In [1]:
import ast
import requests
import pandas as pd
import numpy as np
import sys
sys.path.append("../")
#scr
from scr.infoproducts_zara import get_id_info
from scr.stores import get_stores_zara

In [9]:
product= 7765500250

store = {
    "physicalStoreId":6479
}


stock = "https://itxrest.inditex.com/LOMOServiciosRESTCommerce-ws/common/1/stock/campaign/V2021/product/part-number/{}"
headers ={"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36"}
res = requests.get(stock.format(f"0{product}"), params=store, headers=headers).json()

In [2]:
def get_stock_zara(search_terms,lat,lng):
        
        products = get_id_info(search_terms)

        store_info = get_stores_zara(lat, lng)
        store_ids = [element["id"] for element in store_info]

        stock_url = "https://itxrest.inditex.com/LOMOServiciosRESTCommerce-ws/common/1/stock/campaign/V2021/product/part-number/{}"
        headers ={"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36"}
        stock_params = {"physicalStoreId": store_ids}

        urls = [stock_url.format("".join(p["id_prod"]).rjust(11,"0")) for p in products]
        
        for i,product in enumerate(products):
            headers ={"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36"}
            products[i]["stock"] = requests.get(urls[i],params=stock_params,headers=headers).json()
        #mirar porque no me va el formato json()


In [7]:
get_stock_zara("chaqueta amarilla",40.416050951604056, -3.7072654155715603)

CHALECO DE ESCOTE PICO Y MANGA SISA. CIERRE FRONTAL CON BOTONES.
CHALECO ABIERTO CON MANGA SISA. BOLSILLOS DELANTEROS DE PLASTRÓN.
CHALECO ABIERTO CON MANGA CORTA. CIERRE FRONTAL CON CINTURÓN DEL MISMO TEJIDO.
CHALECO DE ESCOTE PICO Y MANGA SISA. ACABADOS EN RIB. CIERRE FRONTAL CON BOTONES JOYA.

CHALECO ABIERTO CON MANGA CORTA. CIERRE FRONTAL CON CINTURÓN DEL MISMO TEJIDO.
CHALECO CON ESCOTE PICO Y TIRANTES ANCHOS CRUZADOS EN ESPALDA.
CHALECO ABIERTO CON MANGA CORTA. CIERRE FRONTAL CON CINTURÓN DEL MISMO TEJIDO.
CHALECO CON ESCOTE PICO Y TIRANTES ANCHOS CRUZADOS EN ESPALDA.
CHALECO CONFECCIONADO CON TEJIDO EN MEZCLA DE LANA. ESCOTE PICO Y MANGA SISA. DETALLE APLICACIÓN LENTEJUELAS. CIERRE FRONTAL CON BOTONES.
CHALECO ABIERTO CON MANGA CORTA. CIERRE FRONTAL CON CINTURÓN DEL MISMO TEJIDO.

CHALECO CORTO DE CUELLO SOLAPA Y MANGA SISA AMPLIA. BOLSILLOS DE PLASTRÓN CON SOLAPA EN DELANTERO. CIERRE EN DELANTERO CON BOTONES EN RELIEVE A PRESIÓN.

CHALECO CORTO DE CUELLO Y SOLAPA CON MANGA SIS

Funtion to get `all` the info for the products

In [3]:
def products_final_zara(search_item, lat, long):
    
    prod = get_stock_zara(search_item, lat, long)
    
    for product_ in prod:
        print(f"Pincha aquí para comprar el producto:",  product_["url"])
        print(f"Pincha aquí para ver como queda puesto!:", product_["image"])
        print(f"Tipo de producto:", product_["product_name"])
        print(f"Precio:", product_["price"])
    
        for stock in product_["stock"]["stocks"]:
       
            size_stk = []
        
            for sizeStock in stock["sizeStocks"]: 
            
                size_stock = {}
            
          
                size_ = {101:"XS", 102:"S", 103:"M", 104:"L", 105:"XL", 106:"XXL",125:"S-M",131:"L-XL"}
                if sizeStock["size"] in size_:
                    sizeStock["size"] = size_[sizeStock["size"]]
               
                    size_stock["Talla"] = sizeStock["size"]
                else:
                    size_stock["Talla"] = sizeStock["size"]
                
           
                if sizeStock["quantity"] <= 0:
                   size_stock["Stock"] = "Lo siento, no tenemos existencias!☹️"
                elif sizeStock["quantity"] == 1:
                    size_stock["Stock"] = "Últimas existencias!!"
                
                elif sizeStock["quantity"] >= 2:
                    size_stock["Stock"] = "El producto está en tienda!🙂"
                
                elif sizeStock["quantity"] >= 4:
                    size_stock["Stock"] = "Suficientes existencias!!😁"
                
                if size_stock["Stock"] == "No tenemos existencias! 🥲":
                    pass
                else: 
                    size_stk.append(size_stock)
            
        
        
            if len(size_stk) > 0:
                print("id tienda:",stock["physicalStoreId"])
                df = pd.DataFrame(size_stk)
                print(df)

In [5]:
products_final_zara("chaqueta morada", 40.416050951604056, -3.7072654155715603)

CHAQUETA POINTELLE CON ESCOTE PICO Y MANGA LARGA ABULLONADA. CIERRE FRONTAL CON BOTONES PERLA.

CHAQUETA CANALÉ DE CUELLO SOLAPA CON TEJIDO COMBINADO EFECTO PELO DESMONTABLE Y MANGA LARGA. CIERRE FRONTAL CON LAZO.
CHAQUETA CON ESCOTE PICO Y MANGA LARGA CON POMPONES. CIERRE FRONTAL CON BOTONES JOYA.
CHAQUETA CON CUELLO SOLAPA Y MANGA LARGA. CIERRE FRONTAL CON BOTONES FORRADOS A TONO.
CHAQUETA CON CUELLO SOLAPA Y MANGA LARGA. CIERRE FRONTAL CON BOTONES FORRADOS A TONO.
CAZADORA CORTA DE CUELLO SUBIDO CON CAPUCHA AJUSTABLE CON CORDONES ELÁSTICOS Y STOPPER. MANGA LARGA ACABADA EN ELÁSTICO. BOLSILLOS LATERALES CON CREMALLERA OCULTA EN COSTURA. BAJO AJUSTABLE CON CORDONES ELÁSTICOS Y STOPPER. CIERRE FRONTAL CON CREMALLERA TERMOSELLADA.


TypeError: 'NoneType' object is not iterable